# Tale Tree

In [1]:
from enum import Enum

In [ ]:
class Algorithms(Enum):
    RANDOM = None
    BALANCED = None # Balance to avoid sampling too much of the same tree and to keep heterogeneity across lengths of timelines
    TENSION = None

class Event_Type(Enum):
    QUERY = 0
    KEY = 1

class Tension_Level(Enum):
    LOW = 0
    MID = 1
    HIGH = 2

class Event():
    def __init__(self,name:str,tension_level:Tension_Level,type:Event_Type):
        self.name = name
        self.tension_level = tension_level
        self.type = type

class Timeline():
    def __init__(self,timeline:Timeline=None):
        if timeline:
            self.content = timeline.content.copy()
        else:
            self.content = list()

    def add_event(self,event:Event):
        if not self.is_empty() and event.type == self.content[-1].type:
            raise ValueError('Trying to insert an event of same type than the previous one.')
        else:
            self.content.add(event)

    def remove_last_event(self):
        self.content = self.content[:-1]

    def remove_first_event(self):
        self.content = self.content[1:]
    
    def is_empty(self):
        return len(self.content) == 0

def sequence_builder(timelines:list):
    """ Build the sequence of events throught the different timeline. """
    # If a single timeline, then the sequence is straightforward
    if len(timelines) == 1:
        return Timeline(timelines[0])
    # If multiple timelines
    main_sequence = Timeline()
    while not all([t.is_empty() for t in timelines]):
        ...

class Algorithm():
    def __init__(self,timelines:list):
        self.timelines = timelines
    
    def select()

In [19]:
e = Event('test',Tension_Level.HIGH,Event_Type.QUERY)
t = Timeline()